In [ ]:
#Dharnai=0 Berka=1 Madanpur=2
import math
import pandas as pd
import numpy as np
import random
villages = 4
y=20

solar_data = pd.read_csv('Solar.csv')
s1_kw = list(solar_data['s1_kw'])
load_data = pd.read_csv('Load_Data_Villages.csv')
Dharnai_d = list(load_data['Dharnai_d'])
Dharnai_i = list(load_data['Dharnai_i'])
Berka_d = list(load_data['Berka_d'])
Berka_i = list(load_data['Berka_i'])
Madanpur_d = list(load_data['Madan_d'])
Madanpur_i = list(load_data['Madan_i'])
load = [[] for i in range(villages)]
for i in range(y*8760):
    load[0].append(Dharnai_d[i]+Dharnai_i[i])
    load[1].append(Berka_d[i]+Berka_i[i])
    load[2].append(Madanpur_d[i]+Madanpur_i[i])
    load[3].append(0)
    
s1_kw = y*s1_kw
PV_Size = [[56,73,75,77,78,82,83,85,88,90,92,95,97,100,102,106,108,108,108,108],
           [16,18,21,23,26,28,31,33,36,38,41,44,46,49,52,55,57,57,57,57],
           [16,18,19,21,23,24,26,28,29,31,33,35,36,38,39,41,42,42,42,42],
          [0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0]]
Battery_Size = [[111,143,138,152,174,164,187,186,193,200,203,192,218,217,218,220,227,231,232,231],
                [30,56,39,57,52,62,64,86,75,84,89,94,98,95,97,93,95,95,95,95],
                [30,43,61,54,51,61,58,57,68,66,64,65,81,70,77,80,77,78,78,78],
                [111,143,138,152,174,164,187,186,193,200,203,192,218,217,218,220,227,231,232,231]]

Solar_added=[0,0,0,0]
Battery_added=[0,0,0,0]

solar = [[] for i in range(villages)]
eff = [[] for i in range(villages)]
tot_eff = []
life_arr = [[] for i in range(villages)]
rep = [[] for i in range(villages)]
cost = [[] for i in range(villages)]
solar_add_arr=[[] for i in range(villages)]
batt_sub_arr=[[] for i in range(villages)]


def Solar_Data(PV, solar_add, num):
    for n in range(len(s1_kw)):
        solar[num].append((PV[2]+solar_add[num])*s1_kw[n])
    return

Solar_Data(PV_Size[0],Solar_added,0); 
Solar_Data(PV_Size[1],Solar_added,1); 
Solar_Data(PV_Size[2],Solar_added,2);
Solar_Data(PV_Size[3],Solar_added,3);

Battery_Throughput =905
Lifetime_Throughput = [0,0,0,0]
Total_capacity = [0,0,0,0]
capacity_Kw = [0,0,0,0]
Battery_current = [0,0,0,0]
Total_capacity_initial=Total_capacity
decay_constant = [0,0,0,0]
def Battery_Life(Battery, battery_add, num):
    Lifetime_Throughput[num]=(Battery[2]+battery_add[num])*Battery_Throughput
    Total_capacity[num]=(Battery[2]+battery_add[num])*2.4
    Battery_current[num]=(Battery[2]+battery_add[num])*2.4
    capacity_Kw[num]=(Battery[2]+battery_add[num])*2.4
    return
Battery_Life(Battery_Size[0],Battery_added, 0); Battery_Life(Battery_Size[1],Battery_added, 1);
Battery_Life(Battery_Size[2],Battery_added, 2); Battery_Life(Battery_Size[3],Battery_added, 3); 
total_battery = [[] for i in range(villages)]
for i in range(villages):
    total_battery[i].append(Battery_Size[i][2]+Battery_added[i])
soc = [100,100,100,100]
SoC = [[] for i in range(villages)]
kw_Excess = [[] for i in range(villages)]
Percentage_Excess = [[] for i in range(villages)]
Shortage = [[] for i in range(villages)]
battery = [[] for i in range(villages)]
Load_Serve = [[] for i in range(villages)]
E_In = [[] for i in range(villages)]
E_Out = [[] for i in range(villages)]
load_serviced = 0
dod = 15
capacity_percent=[100,100,100,100]
no_of_replacement = [0,0,0,0]
replacement_year_list = [[] for i in range(villages)]
replacement_hour = [0,0,0,0]
replacement_year = [0,0,0,0]
Throughput_hour = [0,0,0,0]
No_of_Replacement = [0,0,0,0]
Replacement_year = [[] for i in range(villages)]
energy_balance = []
Energy_Balance_Battery = Total_capacity
excess=0
canbeserved =[0 for i in range(villages)]
canbeserved_arr=[[] for i in range(villages)]
Battery_Life = [[] for i in range(villages)]

for hour in range(y*8760):
    b=0;c=0;
    battery_current_before = sum(Battery_current)
    load_deficit = [0 for i in range(villages)]
    ex_signal = [0 for i in range(villages)]
    empty = [0 for i in range(villages)]
    soc_dummy = [0 for i in range(villages)]
    cbs_dummy = [0 for i in range(villages)]
    pv_excess_arr = [0 for i in range(villages)]
    pv_hour = [solar[i][hour] for i in range(villages)]
    Energy_In = [0 for i in range(villages)]
    Energy_Out = [0 for i in range(villages)]
    Load_Served = [0 for i in range(villages)]
    shortage = [0 for i in range(villages)]
    Kw_Excess = [0 for i in range(villages)]
    signal =0

    if (hour%24==7):
        pv_pred = [[] for i in range (villages)]
        for num in range(villages):
            bat_dummy = Battery_current[num]
            for h in range (hour,hour+48): 
                if (h<175200):
                    if solar[num][h]>=load[num][h]:
                        pv_pred[num].append(1)
                        e_In=solar[num][h]-load[num][h]
                        e_Out=0
                        bat_dummy = e_In+bat_dummy-e_Out
                    elif solar[num][h]<load[num][h]:
                        pv_pred[num].append(0)
                        if (solar[num][h]+bat_dummy-load[num][h])>=(dod*(Total_capacity[num])/100):
                            e_Out=load[num][h]-solar[num][h]
                            e_In=0
                            bat_dummy=bat_dummy+e_In-e_Out
                        else:
                            e_Out=bat_dummy-(dod*Total_capacity[num])/100
                            e_In=0
                            bat_dummy=bat_dummy+e_In-e_Out
                    if (bat_dummy>capacity_Kw[num]):
                        bat_dummy=capacity_Kw[num]
            canbeserved[num]=(bat_dummy-(Total_capacity[num])*dod/100)

    if (hour<7):
        pv_pred_arr = [0 for i in range(villages)]
    else:
        pv_pred_arr = [pv_pred[i][(hour+17)%24] for i in range(villages)]

    for num in range (villages):
        canbeserved_arr[num].append(canbeserved[num])

    if((all(x == 0 for x in pv_pred_arr) == 1)):
        for num in range (villages):
            if (solar[num][hour]+Battery_current[num]-load[num][hour])>=(dod*(Total_capacity[num])/100):
                Energy_Out[num]=load[num][hour]-solar[num][hour]
                Energy_In[num]=0
                Battery_current[num]=Battery_current[num]+Energy_In[num]-Energy_Out[num]
                load_serviced=load[num][hour]
                Load_Served[num]=(load_serviced)
                shortage[num]=(0)
                load_deficit[num]=0
            else:
                Energy_Out[num]=Battery_current[num]-(dod*Total_capacity[num])/100
                Energy_In[num]=0
                Battery_current[num]=Battery_current[num]+Energy_In[num]-Energy_Out[num]
                load_serviced=solar[num][hour]+Energy_Out[num]
                Load_Served[num]=(load_serviced)
                shortage[num]=(load[num][hour]-load_serviced) 
                load_deficit[num]=(load[num][hour]-load_serviced)        

            Kw_Excess[num]=(0)
            soc[num]=(Battery_current[num]/Total_capacity[num])*100

    elif ((all(x == 1 for x in pv_pred_arr) == 1)):
        for num in range (villages):
            Energy_In[num]=solar[num][hour]-load[num][hour]
            Energy_Out[num]=0
            Battery_current[num]=Energy_In[num]+Battery_current[num]-Energy_Out[num]
            load_serviced=load[num][hour]
            Load_Served[num]=(load_serviced)
            shortage[num]=(0)

            if (Battery_current[num]>capacity_Kw[num]):
                Kw_Excess[num]=(Battery_current[num]- capacity_Kw[num])
                Energy_In[num]=Energy_In[num] - Kw_Excess[num]
                Battery_current[num]=capacity_Kw[num]
                ex_signal[num]=1
            else:
                Kw_Excess[num]=(0)

            soc[num]=(Battery_current[num]/Total_capacity[num])*100

    else:
        for num in range (villages):
            if (solar[num][hour]>load[num][hour]):
                load_serviced=load[num][hour]
                Load_Served[num]=(load_serviced)
                shortage[num]=(0)
                pv_excess_arr[num]=solar[num][hour]-load[num][hour]
                load_deficit[num]=0
            else:
                load_serviced=solar[num][hour]
                Load_Served[num]=(load_serviced)
                shortage[num]=(load[num][hour]-load_serviced)
                pv_excess_arr[num]=0
                load_deficit[num]=load[num][hour]-load_serviced

        for i in range (villages):
                soc_dummy[i] = soc[i]

        while (sum(pv_excess_arr)!=0):
            if ((all(x <= dod for x in soc_dummy) == 1)):
                break
            for i in range (villages):
                soc_dummy[i] = soc[i]
            while (1):
                soc_max_index=soc_dummy.index(max([v for v in soc_dummy if v > dod]))
                if (pv_excess_arr[soc_max_index]>0):
                    break
                else:
                    soc_dummy[soc_max_index]=dod
                if ((all(x <= dod for x in soc_dummy) == 1)):
                    break
            for num in range(villages):
                load_deficit[num]=round(load_deficit[num],2)
            if (sum(load_deficit)==0):
                break
            load_index = load_deficit.index(min([v for v in load_deficit if v > 0]))

            if (pv_excess_arr[soc_max_index]>=load_deficit[load_index]):
                Load_Served[load_index]+=load_deficit[load_index]
                shortage[load_index]=0
                pv_excess_arr[soc_max_index]-=load_deficit[load_index]
                load_deficit[load_index]=0
            else:
                Load_Served[load_index]+=pv_excess_arr[soc_max_index]
                shortage[load_index]-=pv_excess_arr[soc_max_index]
                load_deficit[load_index]=shortage[load_index]
                pv_excess_arr[soc_max_index]=0

        if (sum(load_deficit)==0):
            for num in range (villages):
                Energy_In[num]=pv_excess_arr[num]
                pv_excess_arr[num]=0
                Energy_Out[num]=0
                Battery_current[num]=Energy_In[num]+Battery_current[num]-Energy_Out[num]

                if (Battery_current[num]>capacity_Kw[num]):
                    Kw_Excess[num]=(Battery_current[num]- capacity_Kw[num])
                    Energy_In[num]=Energy_In[num] - Kw_Excess[num]
                    Battery_current[num]=capacity_Kw[num]
                    ex_signal[num]=1
                else:
                    Kw_Excess[num]=(0)

                soc[num]=(Battery_current[num]/Total_capacity[num])*100

        else:
            for num in range (villages):
                if (load_deficit[num]>0):
                    if (Battery_current[num]-load_deficit[num])>=(dod*(Total_capacity[num])/100):
                        Energy_Out[num]=load_deficit[num]
                        Energy_In[num]=0
                        Battery_current[num]=Battery_current[num]+Energy_In[num]-Energy_Out[num]
                        load_serviced=load[num][hour]
                        Load_Served[num]+=load_deficit[num]
                        shortage[num]=0
                        load_deficit[num]=0
                    elif (Battery_current[num]>(dod*Total_capacity[num])/100):
                        Energy_Out[num]=Battery_current[num]-(dod*Total_capacity[num])/100
                        Energy_In[num]=0
                        Battery_current[num]=Battery_current[num]+Energy_In[num]-Energy_Out[num]
                        load_serviced=Energy_Out[num]
                        Load_Served[num]+=(load_serviced)
                        shortage[num]-=(load_serviced) 
                        load_deficit[num]-=Energy_Out[num]
                else:
                    Energy_In[num]=pv_excess_arr[num]
                    pv_excess_arr[num]=0
                    Energy_Out[num]=0
                    Battery_current[num]=Energy_In[num]+Battery_current[num]-Energy_Out[num]

                soc[num]=(Battery_current[num]/Total_capacity[num])*100
                Kw_Excess[num]=(0)  

    while (sum(Kw_Excess)!=0):
        for i in range (villages):
            soc_dummy[i] = round(soc[i],2)
        if (all(x == 100 for x in soc_dummy) == 1):
            break
        if (all(x == 1 for x in ex_signal) != 1):
            for num in range (villages):
                empty[num]=round (capacity_Kw[num]-Battery_current[num],2)
            if (all(x == 0 for x in empty) != 1):
                while (1):
                    soc_min_index=soc_dummy.index(min([v for v in soc_dummy if v >= dod]))
                    if (empty[soc_min_index]==0):
                        soc_dummy[soc_min_index]=100
                    else:
                        break
                    if (all(x == 100 for x in soc_dummy) == 1):
                        break
                if (all(x == 100 for x in soc_dummy) == 1):
                    break
                max_kw_excess=Kw_Excess.index(max([v for v in Kw_Excess if v>0]))
                Energy_In[soc_min_index]+=Kw_Excess[max_kw_excess]
                Battery_current[soc_min_index]+=Kw_Excess[max_kw_excess]
                Kw_Excess[max_kw_excess]=0
                soc[soc_min_index]=(Battery_current[soc_min_index]/Total_capacity[soc_min_index])*100

                if (Battery_current[soc_min_index]>capacity_Kw[soc_min_index]):
                    Kw_Excess[soc_min_index]+=Battery_current[soc_min_index]- capacity_Kw[soc_min_index]
                    Energy_In[soc_min_index]-= Kw_Excess[soc_min_index]
                    Battery_current[soc_min_index]=capacity_Kw[soc_min_index]
                    soc[soc_min_index]=(Battery_current[soc_min_index]/Total_capacity[soc_min_index])*100
                    ex_signal[soc_min_index]=1
            else:
                break
        else:
            break

    for num in range (villages):
        cbs_dummy[num]=canbeserved[num]

    while (sum(load_deficit)!=0):
        if (sum(cbs_dummy)!=0 ):
            for num in range(villages):
                soc[num]=round(soc[num],2)
            if (all(x <=dod for x in soc) == 1):
                break
            if (all(x ==0 for x in cbs_dummy) == 1):
                break
            while (1):
                max_pred_served = cbs_dummy.index(max([v for v in cbs_dummy]))
                if cbs_dummy[max_pred_served]==0:
                    signal = 1
                    break
                if soc[max_pred_served]>dod:
                    break
                else:
                    cbs_dummy[max_pred_served]=0
                if (all(x ==0 for x in cbs_dummy) == 1):
                    break 
            if (signal==1):
                break
            if (all(x ==0 for x in cbs_dummy) == 1):
                break
            for num in range(villages):
                load_deficit[num]=round(load_deficit[num],2)
            if sum(load_deficit)==0:
                break
            min_load_def = load_deficit.index(min([v for v in load_deficit if v>0]))
            if (canbeserved[max_pred_served]>=load_deficit[min_load_def]):
                if (Battery_current[max_pred_served]-load_deficit[min_load_def]) >= (dod*(Total_capacity[max_pred_served])/100):
                    Load_Served[min_load_def]+=load_deficit[min_load_def]
                    Energy_Out[max_pred_served]+=load_deficit[min_load_def]
                    Battery_current[max_pred_served]-=load_deficit[min_load_def]
                    canbeserved[max_pred_served]-=load_deficit[min_load_def]
                    load_deficit[min_load_def]=0
                    shortage[min_load_def]=0
                else:
                    dummy=Battery_current[max_pred_served]-(dod*(Total_capacity[max_pred_served])/100)
                    Load_Served[min_load_def]+=dummy
                    Energy_Out[max_pred_served]+=dummy
                    canbeserved[max_pred_served]-=dummy
                    load_deficit[min_load_def]-=dummy
                    shortage[min_load_def]-=dummy
                    Battery_current[max_pred_served]=(dod*(Total_capacity[max_pred_served])/100)
            else:
                if (Battery_current[max_pred_served]-canbeserved[max_pred_served]) >= (dod*(Total_capacity[max_pred_served])/100):
                    Load_Served[min_load_def]+=canbeserved[max_pred_served]
                    Energy_Out[max_pred_served]+=canbeserved[max_pred_served]
                    Battery_current[max_pred_served]-=canbeserved[max_pred_served]
                    load_deficit[min_load_def]-=canbeserved[max_pred_served]
                    shortage[min_load_def]-=canbeserved[max_pred_served]
                    canbeserved[max_pred_served]=0
                else:
                    dummy=Battery_current[max_pred_served]-(dod*(Total_capacity[max_pred_served])/100)
                    Load_Served[min_load_def]+=dummy
                    Energy_Out[max_pred_served]+=dummy
                    canbeserved[max_pred_served]-=dummy
                    load_deficit[min_load_def]-=dummy
                    shortage[min_load_def]-=dummy
                    Battery_current[max_pred_served]=(dod*(Total_capacity[max_pred_served])/100)
            cbs_dummy[max_pred_served] =  canbeserved[max_pred_served]      
            soc[max_pred_served]=(Battery_current[max_pred_served]/Total_capacity[max_pred_served])*100
        else:
            break

    for i in range(villages):
        battery[i].append(Battery_current[i])
        SoC[i].append(soc[i])
        Load_Serve[i].append(Load_Served[i])
        Shortage[i].append(shortage[i])
        kw_Excess[i].append(Kw_Excess[i])
        E_Out[i].append(Energy_Out[i]) 
        E_In[i].append(Energy_In[i])

    excess= excess + sum(Kw_Excess) 
    e_bal =sum(pv_hour)+battery_current_before-sum(Load_Serve[i][hour] for i in range(villages))-sum(Battery_current)-sum(Kw_Excess)
    energy_balance.append(e_bal)

    for num in range(villages):
        Throughput_hour[num]=round(Throughput_hour[num]+0.39*(Energy_In[num]+Energy_Out[num]),2)
        capacity_percent[num]= round(100-(20*Throughput_hour[num]/Lifetime_Throughput[num]),2) 
        capacity_Kw[num]= round((capacity_percent[num]*Total_capacity[num]/100),2)
        if(Battery_current[num]>capacity_Kw[num]):
            Battery_current[num]=capacity_Kw[num]

        if(Throughput_hour[num]>=Lifetime_Throughput[num]):
            Life=((hour-replacement_hour[num])/8760)
            Battery_Life[num].append(Life)
            replacement_year[num]+=Life
            no_of_replacement[num]+=1
            dummy=int(hour/8760)+(3-int(hour/8760)%3)
            if(dummy>y):
                dummy=y
            del solar[num][hour+1:y*8760]
            for z in range(hour+1,(y*8760)):
                solar[num].append((PV_Size[num][dummy-1]+Solar_added[num])*s1_kw[z])
            Total_capacity[num]=(Battery_Size[num][dummy-1]+Battery_added[num])*2.4
            capacity_Kw[num] = Total_capacity[num]
            Lifetime_Throughput[num]=(Battery_Size[num][dummy-1]+Battery_added[num])*Battery_Throughput
            total_battery[num].append(Battery_Size[num][dummy-1]+Battery_added[num])
            Throughput_hour[num]=0
            soc[num]=100
            Battery_current[num]=Total_capacity[num]
            replacement_hour[num]=hour+1

tot_cost = 0
print ("Total efficiency: ",sum(Load_Serve[0]+Load_Serve[1]+Load_Serve[2])/(sum(load[0])+sum(load[1])+sum(load[2]))*100)
for i in range(villages):
    print ("Village ",i+1)
    T=0; C=0; L=0
    PV_cap = 0
    PO = 0
    PV_add = 0
    PV_total = 0
    if(len(Battery_Life[i])==0):
        L=0
    else:
        L=(sum(Battery_Life[i])+Battery_Life[i][len(Battery_Life[i])-1]-20)
    C=300*(total_battery[i][0])
    O=0;R=0
    rep_c=300
    S=20*(total_battery[i][len(total_battery[i])-1])*L
    Battery_Life[i].append(20-sum(Battery_Life[i]))
    for n in range(len(Battery_Life[i])):
        O=O+(20*((total_battery[i][n])*Battery_Life[i][n]))
        if n>0:
            R=R+((total_battery[i][n])*rep_c)
    T=C+O+R-S
    PV_cap = (PV_Size[i][2]+Solar_added[i])*1500
    PV_add = (PV_Size[i][len(PV_Size[i])-1] - PV_Size[i][2])*1500
    for n in range(len(Battery_Life[i])):
        PO=PO+(500*(PV_Year_Size[i][n]*Battery_Life[i][n]))
    PV_total = PV_cap+PV_add+PO
    tot_cost = tot_cost + PV_total+T
    print ("Total cost: ", PV_total+T)
    if i <3:
        print ("Efficiency: ", (sum(Load_Serve[i])/sum(load[i]))*100)
    print ("No of Replacements: ", no_of_replacement[i])
    #life_arr[i].append(sum(Battery_Life[i]))
print ("Total cost: ", tot_cost)